# Project Two
## Group 5: Brian, Josh, Jeff, Yuliya, Natalie
2016 Election and Police/Judicial Employment

In [9]:
# Import packages
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import os

# (E) EXTRACTION

### Load in police data

In [10]:
# Define directory to police files
police_dir = "Resources/ucr-police-employee-data-2016-by-state-by-counties/"

# Find paths to all police files
police_paths = os.listdir(police_dir)

# Initialize df list
police_df_list = []

# Iterate through each path
for path in police_paths:
    # Identify state of origin
    state = pd.read_excel(police_dir + path, header = 0).iloc[0,0]
    
    # Load in data
    police_df = pd.read_excel(police_dir + path, header = 4)

    # Add state column
    police_df['State'] = state
    
    # Remove '\n' from column names
    police_df = police_df.rename(columns={'Metropolitan/Nonmetropolitan': 'Metropolitan',
                                          'Total law\nenforcement\nemployees': 'Total Law Enforcement Employees',
                                          'Total\nofficers': 'Total Officers',
                                          'Total\ncivilians': 'Total Civilians'})
    
    # Append to list
    police_df_list.append(police_df)

# Merge dataframes together
police_df = pd.concat(police_df_list, axis = 0)
police_df.head()

,Metropolitan,County,Total Law Enforcement Employees,Total Officers,Total Civilians,State
0,Metropolitan Counties,Anoka,256.0,129.0,127.0,MINNESOTA
1,NaN,Benton,70.0,24.0,46.0,MINNESOTA
2,NaN,Blue Earth,69.0,31.0,38.0,MINNESOTA
3,NaN,Carlton,52.0,22.0,30.0,MINNESOTA
4,NaN,Carver,148.0,77.0,71.0,MINNESOTA


### Load in voting and county data

In [11]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///Resources/archive/database.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Create a session
session = Session(engine)

# Create a connection
conn = engine.connect()

# Generate dataframe
voting_df = pd.read_sql_query('select * from primary_results', con = engine)
county_df = pd.read_sql_query('select * from county_facts', con = engine)
county_dict_df = pd.read_sql_query('select * from county_facts_dictionary', con = engine)

# (T) TRANSFORM

### Format Police Data

In [12]:
# Modify values in Metropolitan column to be more readable
police_df = police_df.drop('Metropolitan', axis = 1)

# Modify State to be first capitalized only
police_df['State'] = police_df['State'].str.capitalize()

# Change column names
police_df = police_df.rename({"State":"state_name", "County":"county",
                              "Total Law Enforcement Employees": "total_law_enforcement_employees",
                              "Total Officers": "total_officers",
                              "Total Civilians": "total_civilians"}, axis = 1)
police_df.head()

,county,total_law_enforcement_employees,total_officers,total_civilians,state_name
0,Anoka,256.0,129.0,127.0,Minnesota
1,Benton,70.0,24.0,46.0,Minnesota
2,Blue Earth,69.0,31.0,38.0,Minnesota
3,Carlton,52.0,22.0,30.0,Minnesota
4,Carver,148.0,77.0,71.0,Minnesota


### Format Voting Data

In [13]:
# Make column names consistent
voting_df = voting_df.rename(columns={'state': 'state_name'})

voting_df.head()

### Format County Data

In [14]:
# Remove state and country rows
county_df = county_df[ county_df['state_abbreviation'] != "" ]

# Subset shared states with voting_df for mapping state names
shared_states = list(set(county_df['state_abbreviation']) & set(voting_df['state_abbreviation']))
county_df = county_df[county_df['state_abbreviation'].isin(shared_states)]

# Add full state name column
state_dict = dict(zip(voting_df['state_abbreviation'],voting_df['state_name']))
county_df['state_name'] = county_df['state_abbreviation'].map(state_dict)
county_df.head()

# Subset for columns of interest
columns = ['PST045214', 'POP010210', 'POP060210', 'LND110210', 'PST120214', 
           'EDU635213', 'EDU685213', 'INC910213', 'INC110213']
county_dict_df = county_dict_df[county_dict_df['column_name'].isin(columns)]
county_df = county_df.loc[:,county_df.columns.isin(['area_name', 'fips', 'state_abbreviation', 'state_name',
                                                    columns[0], columns[1], columns[2], columns[3], columns[4],
                                                    columns[5], columns[6], columns[7], columns[8]])]
county_df.head()

# Make shorter column name descriptions
county_dict_df['short_description'] = ['pop_2014', 'pct_pop_delt10to14', 'pop_2010', 'pct_hs_grad', 'pct_post_bach_grad',
                                       'per_capita_income', 'median_household_income', 'sq_miles', 'pop_per_sq_mile']
county_dict_df = county_dict_df.rename({'description':'long_description'}, axis = 1)

# Map column names of county_df to short description
column_name_dict = dict(zip(county_dict_df['column_name'], county_dict_df['short_description']))
county_df = county_df.rename(column_name_dict, axis = 1)

# Remove "County" from county column and rename
county_df = county_df.rename({'area_name':'county'}, axis = 1)
county_df['county'] = county_df['county'].str.replace(" County", "")

### Add Indeces and Export Headers

In [15]:
# Define id column
county_df['id'] = list(range(county_df.shape[0]))
police_df['id'] = list(range(police_df.shape[0]))
county_dict_df['id'] = list(range(county_dict_df.shape[0]))
voting_df['id'] = list(range(voting_df.shape[0]))

# Set id as index
county_df.set_index("id", inplace=True)
police_df.set_index("id", inplace=True)
county_dict_df.set_index("id", inplace=True)
voting_df.set_index("id", inplace=True)

In [16]:
# Export headers to help establish schema
county_df.head().to_csv('Results/county_df.csv')
police_df.head().to_csv('Results/police_df.csv')
county_dict_df.head().to_csv('Results/county_dict_df.csv')
voting_df.head().to_csv('Results/voting_df.csv')

### View headers of each dataset

In [17]:
county_df.head()

,fips,county,state_abbreviation,pop_2014,pct_pop_delt10to14,pop_2010,pct_hs_grad,pct_post_bach_grad,per_capita_income,median_household_income,sq_miles,pop_per_sq_mile,state_name
id,,,,,,,,,,,,,
0,1001,Autauga,AL,55395,1.5,54571,85.6,20.9,24571,53682,594.44,91.8,Alabama
1,1003,Baldwin,AL,200111,9.8,182265,89.1,27.7,26766,50221,1589.78,114.6,Alabama
2,1005,Barbour,AL,26887,-2.1,27457,73.7,13.4,16829,32911,884.88,31.0,Alabama
3,1007,Bibb,AL,22506,-1.8,22915,77.5,12.1,17427,36447,622.58,36.8,Alabama
4,1009,Blount,AL,57719,0.7,57322,77.0,12.1,20730,44145,644.78,88.9,Alabama


In [18]:
police_df.head()

,county,total_law_enforcement_employees,total_officers,total_civilians,state_name
id,,,,,
0,Anoka,256.0,129.0,127.0,Minnesota
1,Benton,70.0,24.0,46.0,Minnesota
2,Blue Earth,69.0,31.0,38.0,Minnesota
3,Carlton,52.0,22.0,30.0,Minnesota
4,Carver,148.0,77.0,71.0,Minnesota


In [19]:
county_dict_df.head()

,column_name,long_description,short_description
id,,,
0,PST045214,"Population, 2014 estimate",pop_2014
1,PST120214,"Population, percent change - April 1, 2010 to ...",pct_pop_delt10to14
2,POP010210,"Population, 2010",pop_2010
3,EDU635213,"High school graduate or higher, percent of per...",pct_hs_grad
4,EDU685213,"Bachelor's degree or higher, percent of person...",pct_post_bach_grad


In [20]:
voting_df.head()

,state_name,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
id,,,,,,,,
0,Alabama,AL,Autauga,1001,Democrat,Bernie Sanders,544,0.182
1,Alabama,AL,Autauga,1001,Democrat,Hillary Clinton,2387,0.800
2,Alabama,AL,Baldwin,1003,Democrat,Bernie Sanders,2694,0.329
3,Alabama,AL,Baldwin,1003,Democrat,Hillary Clinton,5290,0.647
4,Alabama,AL,Barbour,1005,Democrat,Bernie Sanders,222,0.078


# (L) LOAD

### Create database connection

In [21]:
# Create connection
connection_string = "postgres:admin@localhost:5432/ETL_Project_2"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm tables
engine.table_names()

/var/folders/dm/b63slq2n1t98nrhv9m6ztfbm0000gn/T/ipykernel_1608/459572226.py:6: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['county', 'police', 'voting', 'county_dict']

### Load dataframes into the database

In [22]:
# Upload to SQL
county_df.to_sql(name = 'county', con = engine, if_exists = 'append', index = True)
police_df.to_sql(name = 'police', con = engine, if_exists = 'append', index = True)
county_dict_df.to_sql(name = 'county_dict', con = engine, if_exists = 'append', index = True)
voting_df.to_sql(name = 'voting', con = engine, if_exists = 'append', index = True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "county_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO county (id, fips, county, state_abbreviation, pop_2014, pct_pop_delt10to14, pop_2010, pct_hs_grad, pct_post_bach_grad, per_capita_income, median_household_income, sq_miles, pop_per_sq_mile, state_name) VALUES (%(id)s, %(fips)s, %(county)s, %(state_abbreviation)s, %(pop_2014)s, %(pct_pop_delt10to14)s, %(pop_2010)s, %(pct_hs_grad)s, %(pct_post_bach_grad)s, %(per_capita_income)s, %(median_household_income)s, %(sq_miles)s, %(pop_per_sq_mile)s, %(state_name)s)]
[parameters: ({'id': 0, 'fips': 1001, 'county': 'Autauga', 'state_abbreviation': 'AL', 'pop_2014': 55395, 'pct_pop_delt10to14': 1.5, 'pop_2010': 54571, 'pct_hs_grad': 85.6, 'pct_post_bach_grad': 20.9, 'per_capita_income': 24571, 'median_household_income': 53682, 'sq_miles': 594.44, 'pop_per_sq_mile': 91.8, 'state_name': 'Alabama'}, {'id': 1, 'fips': 1003, 'county': 'Ba